In [10]:
import time
import numpy as np
import pandas as pd
import seaborn as sns
import random
import math
import sys 
import copy
import matplotlib.pyplot as plt 
%matplotlib inline
from sklearn.preprocessing import LabelEncoder 
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore') 
from zipfile import ZipFile 
from collections import Counter
import cv2
from collections import deque
import win32gui

HEIGHT = 480
WIDTH = 640
RADIUS = 30
BLACK_COLOR =(0,0,0)
WHITE_COLOR =(255,255,255)
RED_COLOR =(0,0,255)
BLUE_COLOR =(255,0,0)
GREEN_COLOR =(0,255,0)
INITIAL_SPEED = 20

In [11]:
#mathematical function

def line_maker(a , b  ):
    return (a[1]-b[1] , b[0]-a[0] , a[0]*b[1]-a[1]*b[0])

def point_from_line(line , x,y):
    return abs(line[0]*x+line[1]*y+line[2])/math.sqrt(line[0]*line[0]+line[1]*line[1])


def line_from_line(line1 ,  line2 ):
    x = 1
    y = (-line2[0]-line2[2])/line2[1]
    
    return point_from_line(line1 , x, y)


def normal_slope(line):
    return line[1]/line[0] 

def reflected_vector( v , n ):
    ans =  v-(2*np.dot(v,n)/np.square(n).sum())*n
    return ans
    if np.square(ans).sum()<2*INITIAL_SPEED*INITIAL_SPEED:
        ans[0] =math.ceil(ans[0])
        ans[1] =math.ceil(ans[1])
    else:
        ans[0] =math. floor(ans[0])
        ans[1] = math.floor(ans[1])
    return ans


# v =np.array([12,11])
# n = np.array([1,2])

# print(reflected_vector(v,n))



In [21]:

class Ball:
    def __init__(self, radius , color):
        self.x = 100
        self.y = 100
        self.velx = INITIAL_SPEED
        self.vely = INITIAL_SPEED
        self.color = color 
        self.radius = radius 
    def draw(self ,frame  ):
        frame = cv2.circle(frame ,(int(self.x) ,int(self.y)),self.radius , self.color ,-1 )

    def update(self ):#, center , height ,width , angle 
        
        self.x+=self.velx
        self.y+=self.vely
        
        if self.x+self.radius>=WIDTH:
            self.x = WIDTH-self.radius-2
            self.velx = -self.velx
        if self.x-self.radius<=0:
            self.x = self.radius+2
            self.velx = -self.velx
        if self.y+self.radius>=HEIGHT:
            self.y = HEIGHT-self.radius-2
            self.vely = -self.vely
        if self.y-self.radius<=0:
            self.y = self.radius+2
            self.vely = -self.vely
       
        
        
            
      
    def reflection_checker(self ,box):
        new_vel = np.zeros((2));
        lines = []

        lines.append(line_maker(box[0] ,box[1]))
        lines.append(line_maker(box[1] ,box[2]))
        lines.append(line_maker(box[2] ,box[3]))
        lines.append(line_maker(box[3] ,box[0]))

        #one
        a = point_from_line(lines[0] , self.x ,self.y)
        b = point_from_line(lines[2] , self.x ,self.y)
        w = line_from_line(lines[0] ,lines[2])
        c = point_from_line(lines[1] ,self.x ,self.y)
        d = point_from_line(lines[3] ,self.x ,self.y)
        h = line_from_line(lines[1] , lines[3])

        if c+d-RADIUS<=h and b>=w and a<=RADIUS:
            v = np.array([self.velx , self.vely])
            n = np.array([1,normal_slope(lines[0])])
            ans = reflected_vector(v,n)
            self.velx = ans[0]
            self.vely = ans[1]
            return
            
            
        #two
        a = point_from_line(lines[0] , self.x ,self.y)
        b = point_from_line(lines[2] , self.x ,self.y)
        w = line_from_line(lines[0] ,lines[2])
        c = point_from_line(lines[1] ,self.x ,self.y)
        d = point_from_line(lines[3] ,self.x ,self.y)
        h = line_from_line(lines[1] , lines[3])
        
        if c+d-RADIUS<=h and a>=w and b<=RADIUS:
            v = np.array([self.velx , self.vely])
            n = np.array([1,normal_slope(lines[2])])
            ans = reflected_vector(v,n)
            self.velx = ans[0]
            self.vely = ans[1]
            return
            
            
        #three
        a = point_from_line(lines[1] , self.x ,self.y)
        b = point_from_line(lines[3] , self.x ,self.y)
        w = line_from_line(lines[1] ,lines[3])
        c = point_from_line(lines[0] ,self.x ,self.y)
        d = point_from_line(lines[2] ,self.x ,self.y)
        h = line_from_line(lines[0] , lines[2])

        if c+d-RADIUS<=h and b>=w and a<=RADIUS:
            v = np.array([self.velx , self.vely])
            n = np.array([1,normal_slope(lines[1])])
            ans = reflected_vector(v,n)
            self.velx = ans[0]
            self.vely = ans[1]
            return 
        
        #four
        a = point_from_line(lines[1] , self.x ,self.y)
        b = point_from_line(lines[3] , self.x ,self.y)
        w = line_from_line(lines[1] ,lines[3])
        c = point_from_line(lines[0] ,self.x ,self.y)
        d = point_from_line(lines[2] ,self.x ,self.y)
        h = line_from_line(lines[0] , lines[2])

        if c+d-RADIUS<=h and b>=w and a<=RADIUS:
            v = np.array([self.velx , self.vely])
            n = np.array([1,normal_slope(lines[3])])
            ans = reflected_vector(v,n)
            self.velx = ans[0]
            self.vely = ans[1]
            return
    
            
            
            
            
            
                          
        
        
        
        
            
        
        
        
        

In [22]:
cap = cv2.VideoCapture(0)
cap.set(3,500)
cap.set(4,500)
kernel = np.ones((5,5),np.uint8)



ball  = Ball(RADIUS,RED_COLOR)



while True:
    ret , frame= cap.read()
    frame =  cv2.flip(frame ,1)
    hsv = cv2.cvtColor(frame , cv2.COLOR_BGR2HSV)
    lower = np.array([22, 93, 0])
    upper = np.array([45, 255, 255])
    mask = cv2.inRange(hsv , lower , upper)
    mask = cv2.erode(mask ,kernel , iterations=1)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask =cv2.dilate(mask , kernel ,iterations=1)
    cnts ,hei = cv2.findContours(mask , cv2.RETR_EXTERNAL,  cv2.CHAIN_APPROX_NONE)
    cnts = sorted(cnts, key=lambda x: cv2.contourArea(x) ,reverse=True)

    
    
    
    
    
    if len(cnts)>0 and cv2.contourArea(cnts[0])>3000.0 : 

#         cv2.drawContours(frame, cnts[0]   , -1, (0, 255, 0), 3) 
#         x,y,w,h =cv2.boundingRect(cnts[0])
#         cv2.rectangle(frame , (x,y) , (x+w ,y+h),(0,255,0) ,2)


        peri =cv2.arcLength(cnts[0],True)
        approx =  cv2.approxPolyDP(cnts[0],0.02*peri ,True)
        rect= cv2.minAreaRect(cnts[0])
        box = cv2.boxPoints(rect)
        box = np.int0(box)
        cv2.drawContours(frame ,  [box] , 0,BLUE_COLOR ,-1)
        
        center ,(width ,height) , angle = rect
        ball.reflection_checker(box)
 
            
            
 
    
    
    
    
    ball.draw(frame )
    ball.update( )
    
    cv2.imshow('frame', frame)
#     cv2.imshow('mask', mask)
    
          

    k = cv2.waitKey(5)
    if k==27:
        break
cap.release()
cv2.destroyAllWindows()




38
